# Text Summarization

Summarization involves producing a condensed version of a document or article that retains the key information. There are two main types of summarization:

- **Extractive**: Selecting and extracting the most important details directly from the text.
- **Abstractive**: Generating new text that conveys the essential information in a more concise form.

In this project we will:

- Fine-tune the T5 model on a subset of the BillSum dataset containing California state bills for abstractive summarization.
- Run inference using our fine-tuned model.

## Installing Required Libraries

In [1]:
!pip install transformers datasets evaluate rouge_score

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=5bde61ec166216ba9574dcded4b00a1c4a900f4ee00963c480901a4109d20108
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


## Loading The Dataset

In [2]:
from datasets import load_dataset

billsum = load_dataset("billsum", split="ca_test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.27k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/91.8M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/15.8M [00:00<?, ?B/s]

ca_test-00000-of-00001.parquet:   0%|          | 0.00/6.12M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [3]:
billsum = billsum.train_test_split(test_size=0.2)

In [4]:
billsum["train"][0]

{'text': 'The people of the State of California do enact as follows:\n\n\nSECTION 1.\nSection 54964.5 of the Government Code is amended to read:\n54964.5.\n(a) A nonprofit organization or an officer, employee, or agent of a nonprofit organization shall not use, or permit another to use, public resources received from any local agency to make a contribution or expenditure not authorized by law.\n(b) As used in this section and Section 84222.5, the following terms have the following meanings:\n(1) “Local agency” has the same meaning as that term is defined in paragraph (4) of subdivision (b) of Section 54964 and shall also include a public entity created pursuant to the Joint Exercise of Powers Act (Chapter 5 (commencing with Section 6500) of Division 7 of Title 1) by one or more entities described in Section 54964.\n(2) “Nonprofit organization” means an entity incorporated under the Nonprofit Corporation Law (Division 2 (commencing with Section 5000) of Title 1 of the Corporations Code)

we will focus on two key fields:

- **text**: the full content of the bill, serving as the input for the model.
- **summary**: a shortened version of the text, which will be the target output for the model.

## Data Preprocessing

In [5]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [6]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [7]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

In [8]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

## Evaluate

In [9]:
import evaluate

rouge = evaluate.load("rouge")

In [10]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [11]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

## Training The Model

In [12]:
training_args = Seq2SeqTrainingArguments(
    output_dir="output",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    logging_steps=62
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,3.632200,2.809929,0.126300,0.036700,0.105800,0.106200,19.000000
2,2.881000,2.582771,0.137700,0.048400,0.114100,0.114200,19.000000
3,2.718700,2.492684,0.149400,0.056700,0.123100,0.123100,19.000000
4,2.640400,2.441141,0.179500,0.080200,0.149800,0.149500,19.000000
5,2.592600,2.410806,0.190900,0.090800,0.161100,0.161100,19.000000
6,2.552700,2.386251,0.195400,0.095400,0.165800,0.165800,19.000000
7,2.528600,2.372561,0.196300,0.096300,0.166300,0.166300,19.000000
8,2.513100,2.362059,0.196100,0.096300,0.166000,0.166200,19.000000
9,2.499800,2.357382,0.196700,0.097500,0.166400,0.166700,19.000000
10,2.491100,2.355185,0.196700,0.097000,0.166200,0.166400,19.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=620, training_loss=2.705018984886908, metrics={'train_runtime': 706.6332, 'train_samples_per_second': 13.996, 'train_steps_per_second': 0.877, 'total_flos': 2677060833116160.0, 'train_loss': 2.705018984886908, 'epoch': 10.0})

## Save The model

In [13]:
model.save_pretrained('./model')
tokenizer.save_pretrained('./model')

('./model/tokenizer_config.json',
 './model/special_tokens_map.json',
 './model/spiece.model',
 './model/added_tokens.json',
 './model/tokenizer.json')

## Inference

In [14]:
text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."

In [15]:
saved_model_path = "./model"

loaded_tokenizer = AutoTokenizer.from_pretrained(saved_model_path)
inputs = loaded_tokenizer(text, return_tensors="pt").input_ids

In [16]:
loaded_model = AutoModelForSeq2SeqLM.from_pretrained(saved_model_path)
outputs = loaded_model.generate(inputs, max_new_tokens=100, do_sample=False)

In [17]:
loaded_tokenizer.decode(outputs[0], skip_special_tokens=True)

"The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll ask the ultra-wealthy and corporations to pay their fair share."

In [18]:
! zip -r model.zip model

  adding: model/ (stored 0%)
  adding: model/tokenizer.json (deflated 74%)
  adding: model/tokenizer_config.json (deflated 95%)
  adding: model/config.json (deflated 62%)
  adding: model/generation_config.json (deflated 29%)
  adding: model/spiece.model (deflated 48%)
  adding: model/special_tokens_map.json (deflated 85%)
  adding: model/model.safetensors (deflated 11%)
